In [9]:
import pandas as pd

columns = ['birthDate',
           'nationality',
           'id',
           'jerseyNumber',
           'code',
           'type', 
           'abbreviation',
           'birthStateProvince',
           'Team_Number',
           'Unnamed: 0',
           'index',
           'alternateCaptain',
           'captain',
           'active',
           'rookie',
           'rosterStatus',
           'birthCity']

df = pd.read_csv('~/Desktop/NHL-Salary-Predictions/data/cleaned_player_df_dash.csv').drop(columns, axis=1)

# --------------------------- Player Salaries -------------------------------------------------

df = df[df['Salary_2020-21'] != 0.0]
df_datatable_1 = df[['fullName',
                     'Salary_2020-21',
                     'name',
                     'currentAge',
                     'height',
                     'weight',
                     'shootsCatches',
                     'birthCountry']] \
                    .sort_values('Salary_2020-21', ascending=False)
df_datatable_1['shootsCatches'] = df_datatable_1['shootsCatches'].replace('L', 'Left').replace('R', 'Right')
df_datatable_1['Rank'] = range(1, len(df_datatable_1) + 1)
df_datatable_1 = df_datatable_1[['Rank',
                                   'fullName',
                                   'Salary_2020-21',
                                   'name',
                                   'currentAge',
                                   'height',
                                   'weight',
                                   'shootsCatches',
                                   'birthCountry']]
df_datatable_1['id'] = df['fullName']
df_datatable_1.set_index('id', inplace=True)

In [11]:
df_datatable_1['id']

KeyError: 'id'

In [28]:
[col for col in df.columns]

['fullName',
 'name',
 'currentAge',
 'birthCity',
 'birthCountry',
 'height',
 'weight',
 'shootsCatches',
 'rosterStatus',
 'timeOnIce20',
 'assists20',
 'goals20',
 'pim20',
 'shots20',
 'games20',
 'hits20',
 'powerPlayGoals20',
 'powerPlayPoints20',
 'powerPlayTimeOnIce20',
 'evenTimeOnIce20',
 'penaltyMinutes20',
 'faceOffPct20',
 'shotPct20',
 'gameWinningGoals20',
 'overTimeGoals20',
 'shortHandedGoals20',
 'shortHandedPoints20',
 'shortHandedTimeOnIce20',
 'blocked20',
 'plusMinus20',
 'points20',
 'shifts20',
 'timeOnIcePerGame20',
 'evenTimeOnIcePerGame20',
 'shortHandedTimeOnIcePerGame20',
 'powerPlayTimeOnIcePerGame20',
 'ties20',
 'timeOnIce19',
 'assists19',
 'goals19',
 'pim19',
 'shots19',
 'games19',
 'hits19',
 'powerPlayGoals19',
 'powerPlayPoints19',
 'powerPlayTimeOnIce19',
 'evenTimeOnIce19',
 'penaltyMinutes19',
 'faceOffPct19',
 'shotPct19',
 'gameWinningGoals19',
 'overTimeGoals19',
 'shortHandedGoals19',
 'shortHandedPoints19',
 'shortHandedTimeOnIce19',
 'bl

In [54]:
cols = [col for col in df.columns if '14' or 'fullName' in col]

In [55]:
df_14 = df[cols]

In [56]:
df_14[df_14['Salary_2014-15'] != 0.0]

,fullName,name,currentAge,birthCity,birthCountry,height,weight,shootsCatches,rosterStatus,timeOnIce20,...,evenTimeOnIcePerGame14,shortHandedTimeOnIcePerGame14,powerPlayTimeOnIcePerGame14,Salary_2014-15,Salary_2015-16,Salary_2016-17,Salary_2017-18,Salary_2018-19,Salary_2019-20,Salary_2020-21
0,P.K. Subban,Defenseman,32,Toronto,CAN,"6' 0""",210,R,Y,1504:25,...,19:17,00:40,04:39,7000000.0,7000000.0,11000000.0,11000000.0,10000000.0,10000000.0,8000000.0
13,Travis Zajac,Center,36,Winnipeg,CAN,"6' 2""",185,R,Y,1179:36,...,15:48,01:59,02:30,5000000.0,6500000.0,6500000.0,6500000.0,6500000.0,5750000.0,1536639.0
45,Claude Giroux,Center,33,Hearst,CAN,"5' 11""",185,R,Y,1309:57,...,15:44,00:56,03:44,10000000.0,9000000.0,9000000.0,9000000.0,9000000.0,8000000.0,7200000.0
61,Jeff Carter,Center,36,London,CAN,"6' 3""",219,R,Y,1016:05,...,14:10,01:30,03:16,6750000.0,7000000.0,7000000.0,6500000.0,5000000.0,3000000.0,448266.0
62,Sidney Crosby,Center,33,Cole Harbour,CAN,"5' 11""",200,L,Y,831:00,...,17:10,00:30,04:17,12000000.0,12000000.0,10900000.0,10900000.0,10000000.0,9000000.0,9600000.0
80,Patrice Bergeron,Center,35,L'Ancienne-Lorette,CAN,"6' 1""",195,R,Y,1142:16,...,14:02,01:57,01:59,8750000.0,8750000.0,8750000.0,8750000.0,6875000.0,4375000.0,4375000.0
199,Alex Ovechkin,Left Wing,35,Moscow,RUS,"6' 3""",238,R,Y,1405:11,...,15:29,00:00,05:02,10000000.0,10000000.0,10000000.0,10000000.0,10000000.0,10000000.0,10000000.0
295,Gabriel Landeskog,Left Wing,28,Stockholm,SWE,"6' 1""",215,L,Y,1081:35,...,16:03,00:12,02:24,4500000.0,4500000.0,5500000.0,6000000.0,6000000.0,6000000.0,6500000.0
307,Duncan Keith,Defenseman,38,Winnipeg,CAN,"6' 1""",192,L,Y,1487:31,...,19:05,02:23,03:10,7600000.0,7500000.0,6000000.0,5000000.0,4500000.0,3500000.0,2650000.0
345,Ryan Getzlaf,Center,36,Regina,CAN,"6' 3""",228,R,Y,1292:10,...,15:28,02:07,03:40,8750000.0,9250000.0,9250000.0,9025000.0,8950000.0,8275000.0,6000000.0
